In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('./data/screentime_analysis.csv')

print(data.head())

         Date        App  Usage (minutes)  Notifications  Times Opened
0  2024-08-07  Instagram               81             24            57
1  2024-08-08  Instagram               90             30            53
2  2024-08-26  Instagram              112             33            17
3  2024-08-22  Instagram               82             11            38
4  2024-08-12  Instagram               59             47            16


In [ ]:
# drop unnecessary columns
data_gan = data.drop(columns=['Date', 'App'])

# initialize a MinMaxScaler to normalize the data between 0 and 1
scaler = MinMaxScaler()

# normalize the data
normalized_data = scaler.fit_transform(data_gan)

# convert back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=data_gan.columns)

normalized_df.head()